# NPM Kompleks Ag Analizi (Top 200)

Bu defter, populer Top 200 NPM paketini yonlu bir ag (Dependent -> Dependency) olarak modelleyip, in-degree ve betweenness merkeziyet metrikleriyle yapisal riski inceler. Kurulum adimlarini atliyoruz; gerekli paketlerin yüklü oldugu varsayilir.

Contents:
- Veri: `data/top_200.txt` (varsa otomatik kullanilir)
- Ag: NetworkX DiGraph, kenarlar Dependent -> Dependency
- Metrikler: In-degree, Betweenness
- Ciktilar: `results/` altina kaydedilir
- Gorsellestirme: tum ag ve sadece Top 200 indüklenmis alt-ag

Not: Buyuk N (ornegin 20k) icin bagimlilik cekimi ve betweenness hesabi maliyetlidir; bu defter Top 200 icin optimize edilmistir.

## 1) Icel Aktarim ve Parametreler
Yardimci fonksiyonlar `src/analyze_npm_network.py` icindedir. Varsayilan `OUTDIR` = `results/`.

In [ ]:
from pathlib import Path
import sys
import os

# src klasorunu import yoluna ekle
sys.path.append(str(Path('src').resolve()))

import networkx as nx
from analyze_npm_network import (
    fetch_top_packages,
    build_dependency_graph,
    compute_metrics,
    save_edges,
    save_metrics,
    save_report,
    read_list,
)

# Tercihen sabit Top 200 listesi
INPUT_LIST = Path('data/top_200.txt')
USE_INPUT_LIST = INPUT_LIST.exists()
TOP_N = 200  # fallback to fetching if needed
OUTDIR = Path('results')
OUTDIR.mkdir(parents=True, exist_ok=True)
OUTDIR

## 2) Top 200 Paketleri Yukle
`data/top_200.txt` varsa kullan; yoksa API'lerden Top-N cek.

In [ ]:
if USE_INPUT_LIST:
    top_packages = read_list(INPUT_LIST)
    print(f'Loaded {len(top_packages)} from {INPUT_LIST}')
else:
    top_packages = fetch_top_packages(TOP_N)
    print(f'Fetched {len(top_packages)} from API(s)')
len(top_packages), top_packages[:20]

## 3) Yonlu Agi Kur
Dugumler paketler; kenarlar Dependent -> Dependency. Varsayilan olarak sadece `dependencies` kullanilir.

In [ ]:
G, top_set = build_dependency_graph(top_packages)
G.number_of_nodes(), G.number_of_edges()

## 4) Merkeziyet Metrikleri
- In-Degree: Dugume gelen kenar sayisi -> etkilenebilecek alan
- Betweenness: En kisa yollardaki aracilik -> tek hata noktasi riski

In [ ]:
in_deg, btw = compute_metrics(G)
nodes = G.number_of_nodes(); edges = G.number_of_edges()
nodes, edges

## 5) Ilk 15 (Tum Dugumler ve Kohort)
In-degree ve betweenness acisindan liderlere hizli bakis (tum dugumler ve sadece Top 200).

In [ ]:
top_in_all = sorted(in_deg.items(), key=lambda kv: kv[1], reverse=True)[:15]
top_btw_all = sorted(btw.items(), key=lambda kv: kv[1], reverse=True)[:15]
top_in_top = sorted(((n, in_deg.get(n, 0)) for n in top_set), key=lambda kv: kv[1], reverse=True)[:15]
top_btw_top = sorted(((n, btw.get(n, 0.0)) for n in top_set), key=lambda kv: kv[1], reverse=True)[:15]
top_in_all, top_btw_all, top_in_top, top_btw_top

## 6) Sonuclari Kaydet
Kenarlar, metrikler ve kisa rapor `results/` altina yazilir.

In [ ]:
save_edges(G, OUTDIR / 'edges.csv')
save_metrics(in_deg, btw, top_set, OUTDIR / 'metrics.csv')
save_report(in_deg, btw, top_set, OUTDIR / 'report.md')
(OUTDIR / 'top_packages.txt').write_text(os.linesep.join(top_packages), encoding='utf-8')
sorted(OUTDIR.iterdir())

## 7) Hizi Dogrulama
Rastgele bazi paketler icin registry'deki dependencies ile graf kenarlarini karsilastir. Surum farklarindan kaynakli kucuk farkliliklar olabilir.

In [ ]:
import random, requests, urllib.parse
def get_registry_deps(name: str):
    url = f'https://registry.npmjs.org/{urllib.parse.quote(name, safe="")}'
    r = requests.get(url, timeout=30)
    if r.status_code != 200:
        return {}
    data = r.json()
    latest = (data.get('dist-tags') or {}).get('latest')
    versions = data.get('versions') or {}
    obj = versions.get(latest) if latest in versions else {}
    return obj.get('dependencies') or {}

sample = random.sample(top_packages, k=min(5, len(top_packages)))
mismatches = {}
for pkg in sample:
    deps_live = set(get_registry_deps(pkg).keys())
    deps_graph = set(v for u,v in G.out_edges(pkg))
    if deps_live != deps_graph:
        mismatches[pkg] = {
            'live_only': sorted(list(deps_live - deps_graph)),
            'graph_only': sorted(list(deps_graph - deps_live)),
        }
mismatches

## 8) Tum Ag Cizimi (Top 200 + Bagimliliklar)
Top 200 dugumleri turuncu, digerleri mavi; dugum boyutu in-degree ile olceklenir.

In [ ]:
import math
try:
    import matplotlib.pyplot as plt
    N = G.number_of_nodes()
    pos = nx.spring_layout(G, k=1/math.sqrt(max(N,1)), seed=42)
    max_in = max(in_deg.values()) if in_deg else 1
    node_sizes = [100 + (400 * (in_deg.get(n,0) / max_in if max_in else 0)) for n in G.nodes()]
    node_colors = ['tab:orange' if n in top_set else 'tab:blue' for n in G.nodes()]
    plt.figure(figsize=(12, 9), dpi=150)
    nx.draw_networkx_edges(G, pos, arrows=False, width=0.5, alpha=0.25)
    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, alpha=0.85, linewidths=0.2, edgecolors='#444444')
    plt.title('NPM Top 200 Dependency Network (Full)')
    plt.axis('off')
    out_png = OUTDIR / 'network_full_top200.png'
    out_svg = OUTDIR / 'network_full_top200.svg'
    plt.tight_layout(); plt.savefig(out_png); plt.savefig(out_svg)
    (out_png, out_svg)
except ModuleNotFoundError:
    print('matplotlib not found; skipping drawing.')


## 9) Sadece Top 200 Induklenmis Alt-Ag
Sadece Top 200 dugumleri; okunabilirlik icin az sayida dugum varsa etiketler gosterilir.

In [ ]:
try:
    H = G.subgraph(top_set).copy()
    HN = H.number_of_nodes() or 1
    posH = nx.spring_layout(H, k=1/math.sqrt(HN), seed=42)
    max_in_H = max((in_deg.get(n,0) for n in H.nodes()), default=1)
    node_sizes_H = [120 + (480 * (in_deg.get(n,0) / max_in_H if max_in_H else 0)) for n in H.nodes()]
    node_colors_H = ['tab:orange' for _ in H.nodes()]
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 8), dpi=150)
    nx.draw_networkx_edges(H, posH, arrows=False, width=0.6, alpha=0.35)
    nx.draw_networkx_nodes(H, posH, node_size=node_sizes_H, node_color=node_colors_H, alpha=0.9, linewidths=0.2, edgecolors='#444444')
    if H.number_of_nodes() <= 60:
        nx.draw_networkx_labels(H, posH, font_size=6)
    plt.title('NPM Top 200 Dependency Network (Top-Only)')
    plt.axis('off')
    out_png2 = OUTDIR / 'network_top200_only.png'
    out_svg2 = OUTDIR / 'network_top200_only.svg'
    plt.tight_layout(); plt.savefig(out_png2); plt.savefig(out_svg2)
    (out_png2, out_svg2)
except ModuleNotFoundError:
    print('matplotlib not found; skipping drawing.')


## 10) Varsayimlar ve Sinirlamalar
- Kenar yonu Dependent -> Dependency; yayilim analizi icin uygundur.
- Yalnizca `dependencies` dahil; `peerDependencies` varsayilan olarak disarida.
- Global dependent sayilari dahil degil; ecosyste.ms verisi eklenebilir.
- En guncel surum kullanilir; eski surumlerde bagimliliklar farkli olabilir.